# **Atividade Prática**
Data: 16/04/2021
### Envolve Pré-processamento e os métodos de classificação kNN e Naive Bayes

### **Detalhes da atividades**

O objetivo desta atividade é você colocar em prática o que estudou sobre pré-processamento e comparar o desempenho (em termos de acurácia) dos algoritmos kNN e Naive Bayes.

### Nesta atividade nós utilizaremos a base de dados ***Titanic***. 
---
![texto alternativo](https://drive.google.com/uc?export=view&id=1-1jHY_FrynJy-k8GLJ0F5PxBN0SyUOVP) 
---
O objetivo é fazer uma predição se um passageiro sobreviverá ou não. A predição será baseada em atributos como:  idade, gênero, classe, local de embarque, etc.

### Obtendo a base
Você deve acessar o site da [Kaggle](https://www.kaggle.com/), [Desafio Titanic ](https://www.kaggle.com/c/titanic) e fazer o download de `train.csv` e `test.csv`. 

Na célula abaixo, você deve carregar a base de dados utilizando Pandas. 
A base de dados já está dividida em um conjunto de treino (training set) e um conjunto de teste (test set). 

In [1]:
import pandas as pd

## Continue aqui para carregar a base de treino e a base de teste

train_data = pd.read_csv('/home/mariohn/Documentos/VSCODE/Datasets/train.csv')
test_data =  pd.read_csv('/home/mariohn/Documentos/VSCODE/Datasets/test.csv')

É importante observar que a base de teste *não* contém os rótulos. Como este problema é um desafio do Kaggle, o objetivo é treinar o melhor modelo possível utilizando a base de treinamento, fazer as predições na base de teste, e submeter essas predições ao site da competição do Kaggle para descobrir o score final. Nesta atividade, entretanto, a submissão das predições ao site da competição não é solicitada.

O comando abaixo mostra as linhas iniciais da base de treinamento:

In [2]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Como as instâncias estão distribuídas entre as partições?
print("Particao de Treino", train_data.shape)
print("Particao de Teste", test_data.shape)

Particao de Treino (891, 12)
Particao de Teste (418, 11)


A partição de treino tem **891 instâncias** e **11 atributos** (constam 12 porque um deles é o rótulo da instância), enquanto a base de teste tem **418 instâncias** e, obviamente, a mesma quantidade de atributos.

Os atributos têm o seguinte significado:
* **Survived**: Essa é a classe (alvo), 0: não sobreviveu; 1: sobreviveu.
* **Pclass**: É a classe do passageiro, isto é, se o passageiro estava na primeira classe, na segunda, etc.
* **Name**, **Sex**, **Age**: autoexplicativo
* **SibSp**: quantos irmãos e esposas/maridos do passageiro estavam a bordo do Titanic.
* **Parch**: quantos filhos e pais do passageiro estavam a bordo do Titanic.
* **Ticket**: id do bilhete
* **Fare**: preço pago (em Libras)
* **Cabin**: número da cabine do passageiro
* **Embarked**: local de embarque do passageiro

Vamos verificar a divisão das instâncias por classe (0 ou 1):

In [4]:
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

A base está ligeiramente desbalanceada, ou seja, há mais pessoas na classe 0 do que na classe 1. Isso é óbvio, dado o problema analisado.

Vamos verificar se há **valores ausentes**:

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Como pode ser observado, os atributos **Age**, **Cabin** e **Embarked** apresentam valores ausentes (*null*), pois esses atributos têm menos de 891 valores *non-null*. 
- O atributo **Cabin** é o que mais sofre com ausência de atributos (77% são *null*).  
- O atributo **Age** tem cerca 19% de valores ausentes.
---
* Portanto, será necessário fazer a imputação dos valores ausentes. Por exemplo, substituir os valores *null* pela idade média no atributo **Age**  parece bem razoável.

* É importante lembrar que o classificador Naive Bayes funciona muito bem com valores ausentes, ou seja, não é necessário fazer imputação de valores ausentes. Porém, esse processo será feito para facilitar a comparação entre Naive Bayes e kNN.

Vamos ver detalhes de alguns dos **atributos categóticos**.

In [6]:
train_data["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [7]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [8]:
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Os valores do atributo **Embarked** são os seguintes: C=Cherbourg, Q=Queenstown, S=Southampton.

Os atributos numéricos são detalhados no comando da célula abaixo:


In [10]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## **Exercício**:
 Realize o **pipeline** de pré-processamento dos dados para que a base seja composta apenas por atributos numéricos. Importante: o pré-processamento deve ser feito somente nos atributos - não incluir o rótulo da instância, isto é, a coluna **Survived**.

1.   Substitua os valores ausentes dos atributos categóricos pelo valor mais frequente.
2.   Substitua os valores ausentes dos atributos numéricos pela média dos valores presentes.
3. Utilize OneHotEncoder

- Você pode utilizar `Pipeline`, `SimpleImputer` , etc.

 

In [18]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


In [19]:
categorical_features = ['Sex', 'Embarked']
numeric_features = ['Age', 'Fare', 'Pclass', 'SibSp', 'Parch']

In [20]:
# Criando as etapas do pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [21]:
# Combinando as etapas em um pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [28]:
X = train_data.drop('Survived', axis=1)
y = train_data['Survived']

X_processed = pipeline.fit_transform(X)

#Classificação
---
Nesta etapa nós utilizaremos a base de treinamento para comparar kNN com Naive Bayes.
Nós vamos trabalhar com duas partes da base: X_train (contendo as instâncias de treinamento), e y_train, que terá os rótulos das instâncias de treinamento. Por exemplo, veja a criação de y_train abaixo.

In [35]:
#Gerando os rótulos da base de treinamento
y_train = train_data["Survived"]
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [36]:
X_train = X_processed
X_train

array([[-0.56573646, -0.50244517,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.66386103,  0.78684529, -1.56610693, ...,  1.        ,
         0.        ,  0.        ],
       [-0.25833709, -0.48885426,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.1046374 , -0.17626324,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       [-0.25833709, -0.04438104, -1.56610693, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.20276197, -0.49237783,  0.82737724, ...,  0.        ,
         1.        ,  0.        ]])

Para facilitar a comparação entre os dois métodos de classificação, nós vamos dividir a base de treinamento em duas partições: treino e validação. Dessa forma, nós treinaremos os modelos na partição de treino e avaliaremos o modelo na partição de validação.

In [37]:
from sklearn.model_selection import train_test_split

#Dividindo a base (com seleção de instâncias aleatória)
X_treino, X_val, y_treino, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state = 42)

#Checando a divisão das instâncias entre as duas bases
print(X_treino.shape, y_treino.shape)
print(X_val.shape, y_val.shape)


(712, 10) (712,)
(179, 10) (179,)


Chamada dos métodos para uso dos algoritmos de classificação

In [38]:
# Chamando os métodos que serão utilizados (kNN e Naive Bayes)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Para avaliação
from sklearn.metrics import accuracy_score

Treinando e testando o Naive Bayes

In [39]:
#Naive Bayes : gerando o modelo

NB_clf = GaussianNB()

# Treinando, testando e avaliando
NB_clf.fit(X_treino, y_treino)
y_pred_NB = NB_clf.predict(X_val)
print(accuracy_score(y_val, y_pred_NB))

0.776536312849162


Treinando e testando o kNN

In [40]:
#kNN : gerando o modelo com número de vizinhos igual a 1

kNN_clf = KNeighborsClassifier(n_neighbors=1)

# Treinando, testando e avaliando
kNN_clf.fit(X_treino, y_treino)
y_pred_kNN = kNN_clf.predict(X_val)
print(accuracy_score(y_val, y_pred_kNN))

0.770949720670391


## Atividades

1.   Teste o classificador kNN variando o valor do parâmetro k. Você deve testar até o valor k = 9




In [41]:
#kNN : gerando o modelo com número de vizinhos igual a 9

kNN_clf_2 = KNeighborsClassifier(n_neighbors=9)

# Treinando, testando e avaliando
kNN_clf_2.fit(X_treino, y_treino)
y_pred_kNN = kNN_clf_2.predict(X_val)
print(accuracy_score(y_val, y_pred_kNN))

0.7988826815642458


2.  Qual valor de k produzir maior acurácia?
3.  Qual algoritmo obteve maior acurácia: kNN ou Naive Bayes?



OBS: Se você desejar submeter a sua resposta ao Kaggle, você deverá executar o pré-processamento na partição de teste e calcular a predição do classificador na base de teste. Por exemplo, supondo que kNN tenha obtido melhor resultado, você deve treinar o kNN com a base de treino completa (X_train), em seguida, você executa o pipeline de pré-processamento na base de teste e calcula a predição do classificador na base de teste. Por exemplo, veja o código a seguir:


In [42]:
kNN_clf_3 = KNeighborsClassifier(n_neighbors=9)

# Treinando, testando e avaliando
kNN_clf_3.fit(X_train, y_train)
X_test = pipeline.fit_transform(test_data)
y_pred = kNN_clf_3.predict(X_test)

Na sequência, você pode criar um arquivo CSV  com  essas predições (respeitando o formato esperado pelo Kaggle), e fazer o upload do mesmo para verificar o desempenho do seu classificador. 